## Career Recommendation Model Using Deep Learning.

a career recommendation model that uses both content based and collaborative filtering to make its recommendations
the model takes in a user profile and does collaborative filtering to find careers that the user has high probability of success in and passes that career path to the content based filter to find other paths that are similar to that one. the final output is the three career 1 from collaborative filtering and the other two from content based filtering.

In [1]:
# necessary libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
df_coll = pd.read_excel("collaborative_filtering.xlsx")
df_coll.head()

,Employee Id,Employee Name,Department,Position,Experience,Performance,Education,Certification,Manager's Feedback,Eligibility
0,101,Michael Miller,IT,Employee,5,Excellent,MSC,3,Excellent,1
1,102,Carl Fox,Production,HR Assistant,9,Low,IT,6,Adaptive,0
2,103,Robert Johnson,Marketing,Senior Manager,3,Low,MSC,2,Moderate,0
3,104,Raymond Zimmerman,IT,Supervisor,7,Low,MSC,10,Not Flexible,0
4,105,Christopher Riddle,Accounts,HR Assistant,5,Average,MSC,10,Extraordinary,1


In [3]:
df_coll.Position.value_counts()

Position
HR Assistant       87
HR Manager         78
General Manager    75
Supervisor         66
Senior Manager     65
Team Leader        65
Employee           64
Name: count, dtype: int64

## Collaborative Filtering

## Content based Filtering

In [4]:
content_data = pd.read_csv("career_paths.csv")

# combining interests and skills
content_data['combined_features'] = content_data['Interest'] + ' ' + content_data['Skills']

# TF-IDF Vectorisation
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(content_data['combined_features'])

# Calculating similarity scores
content_similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)

# function to get content based recommendations
def content_based_recommendation(user_profile, content_similarity_matrix, num_recommendations=2):
    user_vector = vectorizer.transform([user_profile])
    scores = linear_kernel(user_vector, tfidf_matrix).flatten()

    # finding indices of top similarity scores
    top_field_indices = scores.argsort()[-num_recommendations:][::-1]
    recommended_fields = content_data['Field'].iloc[top_field_indices].to_list()

    return recommended_fields


interests = input("enter interests: ")
skills = input("enter skills: ")
additional_info = input("Enter additional info: ")

user_profile = f"{interests} {skills} {additional_info}"
recommended_fields = content_based_recommendation(user_profile, content_similarity_matrix)
print(recommended_fields)

['mathematics', 'english']


In [5]:
# loading pickle
with open('vectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)


with open('similarity_matrix.pickle', 'wb') as f:
    pickle.dump(content_similarity_matrix, f)


import cloudpickle

# Use cloudpickle since linear_kernel might not be picklable directly
with open('recommendation_function.pickle', 'wb') as f:
    cloudpickle.dump(content_based_recommendation, f)
